In [2]:
import numpy as np

In [1]:
Ec_area = 0.49
Ej_over_area = 48.5
El_per_junction = 35.4

In [3]:
def create_matrix_R(N):
    # Create an NxN identity matrix
    R = np.eye(N)

    # Subtract 1 from the diagonal just above the main diagonal
    for i in range(N-1):
        R[i, i+1] = -1

    # Replace the last row with ones
    R[-1, :] = 1

    return R

In [4]:
# Example usage with N=5
N = 5
R_matrix = create_matrix_R(N)
R_matrix

array([[ 1., -1.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0.],
       [ 0.,  0.,  0.,  1., -1.],
       [ 1.,  1.,  1.,  1.,  1.]])